In [18]:
import json
import numpy as np
from bottom_up.all import pick_bfs_connected
from bottom_up.all import get_lr
from bottom_up.all import get_dependents
from bottom_up.all import get_f1
from bottom_up.all import featurize_ultra_local
from bottom_up.all import get_local_feats
from bottom_up.all import get_global_feats
from bottom_up.all import featurize_parent_proposal
from bottom_up.all import pick_l2r_connected
from bottom_up.all import featurize_child_proposal
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction import DictVectorizer
from code.printers import pretty_print_conl
from bottom_up.all import pick_bfs_connected
from bottom_up.all import len_tree
%load_ext autoreload

%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [8]:
import pickle
with open("a", "rb") as inf:
    dt = pickle.load(inf)
    
clf, vectorizer = dt

/Users/ahandler/.virtualenvs/p3/lib/python3.6/site-packages/sklearn/base.py:311: UserWarning: Trying to unpickle estimator LogisticRegression from version 0.20.2 when using version 0.19.1. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)
/Users/ahandler/.virtualenvs/p3/lib/python3.6/site-packages/sklearn/base.py:311: UserWarning: Trying to unpickle estimator DictVectorizer from version 0.20.2 when using version 0.19.1. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)


In [23]:
def pick_at_random(l, **kwargs):
    return random.sample(l, 1)[0]

from code.treeops import bfs
from bottom_up.all import pick_bfs
from bottom_up.all import get_dependents_and_governors
import random

def runtime_path_wild_frontier(sentence, pi, clf, vectorizer, verbose=False):
    T = {i for i in sentence["q"]}
    F = set()
    
    # init frontier
    for v in T:
        for i in sentence["tokens"]:
            F.add(i["index"])

    d, heads, c = bfs(g=sentence, hop_s=0) 
    while len(F) > 0:
        vertex = pi(F=F, d=d, T=T, s=sentence)

        if vertex != 0: # bug here?
            feats = get_local_feats(vertex=vertex, sentence=sentence, d=d, current_tree=T)
            feats = get_global_feats(sentence=sentence, feats=feats, vertex=vertex, current_tree=T)

            X = vectorizer.transform([feats])
            pred = clf.predict_proba(X)[0]
            y = clf.predict(X)[0]

            if y == 1:
                T.add(vertex)
                if vertex != 0:
                    for i in get_dependents_and_governors(vertex, sentence, T):
                        if i not in T and i is not None:
                            F.add(i)
                else:
                    for i in get_dependents(sentence, vertex):
                        if i["dependent"] not in T and i is not None:
                            F.add(i)
        F.remove(vertex)

    return T

tot = 0
for pno, paths in enumerate(validation_paths):
    paths=json.loads(paths)
    sentence = paths["sentence"]
    predicted = runtime_path_wild_frontier(sentence, pi=pick_l2r_connected, clf=clf, vectorizer=vectorizer)
    f1s = get_f1(predicted, sentence)
    tot += f1s
print(tot/len(validation_paths))

0.8234226466815519


In [ ]:
sentence = paths["sentence"]

In [ ]:
from bottom_up.all import get_labels_and_features
from bottom_up.all import get_governor

import json
import pickle
import numpy as np
from bottom_up.all import get_lr
from bottom_up.all import get_dependents
from bottom_up.all import get_f1
from bottom_up.all import featurize_ultra_local
from bottom_up.all import featurize_parent_proposal
from bottom_up.all import featurize_child_proposal
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction import DictVectorizer
from code.printers import pretty_print_conl
from bottom_up.all import len_tree

training_paths = [_ for _ in open("training.paths")]
validation_paths = [_ for _ in open("validation.paths")]


vectorizer = DictVectorizer(sparse=True)

train_features, train_labels = get_labels_and_features(training_paths)

X_train = vectorizer.fit_transform(train_features)

y_train = np.asarray(train_labels)

val_features, val_labels = get_labels_and_features(validation_paths)

X_val = vectorizer.transform(val_features)

y_val = np.asarray(val_labels)

clf = LogisticRegression(random_state=0,
                         solver='lbfgs',
                         C=.5,
                         multi_class='ovr').fit(X_train, y_train)

print(clf.score(X_val,y_val))
print(clf.score(X_train, y_train))

In [131]:
confusion = []
from collections import Counter
from collections import defaultdict
all_vals = defaultdict(list)
for i in np.where(clf.predict(X_val) != y_val)[0]:
    try:
        if train_features[i]["dep"] == "det":
            print(y_val[i], train_features[i]['dependentGloss'], train_features[i], train_features[i]['governorGloss'])
        confusion.append(train_features[i]['dep'])
    except KeyError:
        pass

Counter(confusion).most_common(10)


0 a {'dep': 'det', 'governorGloss': 'drama', 'dependentGloss': 'a', 'type': 'CHILD', 'position': 0.30434782608695654, 'det_gov_pos': False, 'parent_label': 'detdrama', 'child_label': 'deta', 'ner': 'O', 'pos': 'DT', 'depth': 4, 'disconnected': 0, 'over_r': False, 'remaining': 16} drama
0 An {'dep': 'det', 'governorGloss': 'man', 'dependentGloss': 'An', 'type': 'CHILD', 'position': 0.047619047619047616, 'det_gov_pos': False, 'parent_label': 'detman', 'child_label': 'detAn', 'ner': 'O', 'pos': 'DT', 'depth': 3, 'disconnected': 0, 'over_r': False, 'remaining': 45} man
0 a {'dep': 'det', 'governorGloss': 'investment', 'dependentGloss': 'a', 'type': 'CHILD', 'position': 0.6206896551724138, 'det_gov_pos': False, 'parent_label': 'detinvestment', 'child_label': 'deta', 'ner': 'O', 'pos': 'DT', 'depth': 5, 'disconnected': 0, 'over_r': False, 'remaining': 20} investment
1 an {'dep': 'det', 'governorGloss': 'forum', 'dependentGloss': 'an', 'type': 'CHILD', 'position': 0.39285714285714285, 'det_go

[('punct', 100),
 ('compound', 76),
 ('nmod', 73),
 ('amod', 55),
 ('det', 45),
 ('case', 45),
 ('dobj', 33),
 ('aux', 31),
 ('mark', 27),
 ('advcl', 20)]